## Evaluation:
Extraktion von Wortpaaren ergänzt durch Terme

Die Datensätze für die Evaluierung bestehen aus den Beschriftungen und Kontexttexte der 48 Abbildungen aus Artikel, die aus Open Access Journals (NOA Projekt http://noa.wp.hs-hannover.de) und den geleichen Abbildungen, die vor Erstellung dieser Arbeit, bei Wikimedia Commosn vorhanden waren. Es sind 48 einzelne txt-Dateien. 

Kookkurenzen vom Wörter im gleichen Satz.
Lemmata statt Wörter.

Aus den Wortpaaren die Terme NN, VP, NNS. Zusätzlich  NN, VB, NNS, NP und NNP, die nicht Teil einer Nominalphrase sind.

In [2]:
import glob
import nltk
import codecs
import re
import treetaggerwrapper
from nltk.corpus import stopwords

swlist = stopwords.words('english')
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
filelist = glob.glob("evalu-commons/*.txt") #Datensätze für Evaluierung
nr_of_words = 0
fdist = nltk.FreqDist() #Worthäufigkeiten Anzahl
fdist2 = nltk.FreqDist() #Wortpaarhäufigkeiten Frequenz von Wortpaaren

for datei in filelist:  
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text,language='english')#Texte aufteilen, Jeder Satz ist eine Liste von Tokens
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]#Daraus Wortpaare und Wörter extrahieren
    words = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True)
        tags2 = treetaggerwrapper.make_tags(tags);
        words = [lemma for (word,pos,lemma) in tags2 if pos[0]]#Jede Wortart+ Lemma
        word_pairs = [] #Wortpaare
        for i in range(len(tags2)-1) :#Bereich Liste von Zahlen 0 bis Länge von "tags2" -1. Das nächste letzte Wort soll immer angeschaut werden.
            t1 = tags2[i]# Wort i + i+1, 
            t2 = tags2[i+1]
            if t1.pos[0] and t2.pos[0]:
                l1 = t1.lemma #daraus werden die Lemma gebildet
                l2 = t2.lemma
                if l1 not in swlist and l2 not in swlist: #Stopwörter rausnehmen
                    word_pairs.append((l1,l2)) #häufigsten Wortpaare. Nur Häufigkeit.
                
        nr_of_words += len(words)
        fdist.update(words)
        fdist2.update(word_pairs)
        
    
print(fdist2.most_common(3))

[((')', '.'), 225), (('(', 'Figure'), 145), ((',', '@card@'), 133)]


## Eingrenzung auf ein Muster

Wortartmuster für Kookkurrenzen (Wortartmuster für Nominalphrasen) 

In [4]:
import glob
import nltk
import codecs
import re
import treetaggerwrapper
import pprint
import math
from nltk.corpus import stopwords

swlist = stopwords.words('english')
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
filelist = glob.glob("evalu-commons/*.txt")
nr_of_words = 0
fdist = nltk.FreqDist() #Worthäufigkeiten
fdist2 = nltk.FreqDist() #Wortpaarhäufigkeiten


npmuster = [('NN','NN'), #Nomen
            ('NP','NP'), #Eigennamen
            ('NN','NP'), 
            ('NP','NN'),
            ('JJ','NP'), # Adjektive
            ('JJ','NN')
           ]
    

for datei in filelist:  
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()

    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    words = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        words = [lemma for (word,pos,lemma) in tags2 if pos[0]]
        word_pairs = []
        for i in range(len(tags2)-1) :
            t1 = tags2[i]
            t2 = tags2[i+1]
            if (t1.pos,t2.pos) in npmuster:
                l1 = t1.lemma
                l2 = t2.lemma
                if l1 not in swlist and l2 not in swlist  and len(l1) >2 and len(l2) >2: #Englische Stopwörter
                    word_pairs.append((l1,l2))
                    
        nr_of_words += len(words)
        fdist.update(words)
        fdist2.update(word_pairs)
print("Anzahl Wortvorkommen (tokens): ", nr_of_words)
pprint.pprint(fdist2.most_common(100)) 



Anzahl Wortvorkommen (tokens):  15971
[(('bipartite', 'inhibitor'), 26),
 (('crystal', 'structure'), 22),
 (('electron', 'density'), 19),
 (('dmaDap', 'sidechain'), 16),
 (('2—figure', 'supplement'), 15),
 (('transcription', 'initiation'), 14),
 (('structural', 'model'), 13),
 (('violet', 'sphere'), 11),
 (('transcription', 'elongation'), 11),
 (('nucleotide', 'addition'), 10),
 (('Crystal', 'structure'), 10),
 (('elongation', 'complex'), 9),
 (('atomic', 'model'), 8),
 (('steric', 'clash'), 8),
 (('wild-type', 'RNAP'), 8),
 (('RNAP-Rif', 'interaction'), 8),
 (('Rif', 'target'), 8),
 (('bridge', 'helix'), 7),
 (('4—figure', 'supplement'), 7),
 (('1—figure', 'supplement'), 7),
 (('transcription', 'inhibition'), 7),
 (('blue', 'mesh'), 7),
 (('contrast', 'agent'), 6),
 (('common', 'ancestor'), 6),
 (('high', 'potency'), 6),
 (('10.7554/eLife.02450', 'relationship'), 6),
 (('gray', 'ribbon'), 6),
 (('Figure', '1—figure'), 6),
 (('i+1', 'site'), 6),
 (('Mason', 'reference'), 6),
 (('struct

## Extraktion von Nominalphrasen aus einzelne Datensätze

Nominalphrasen nach Muster für Extraktion verwenden.
Tokenisierung ändern: Häufigkeit und tf.idf Berechnung (Nominalphrase als einziges Token)

Aus den Wortpaaren die Terme NN, VP, NNS. Zusätzlich  NN, VB, NNS, NP und NNP, die nicht Teil einer Nominalphrase sind.

In [5]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper

df = {} 
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
NPList = fdist2.keys()

def candidates(taglist):#Liste mit Tags, als Ausgabe als Liste
    cand = []
    skip = False
    for i in range(len(taglist)-1) :
        if skip:
            skip = False
            continue
        skip = False
        l1 = taglist[i].lemma # erstes Wort
        l2 = taglist[i+1].lemma# zweites Wort
        if len(l1) >2 and len(l2) >2:   #Die einzelnen Wörter der NP sollen eine Länge von über 2 Buchstaben haben
            if (l1,l2) in NPList:
                skip = True
                cand.append((l1,l2))
                if taglist[i].pos == 'NN' or taglist[i].pos == 'NP': #Wenn im Wortpaar ein NN oder NP vorkommt, als Term dazu
                    cand.append((l1))
            else:
                if taglist[i].pos == 'NN' or taglist[i].pos == 'VB' or taglist[i].pos == 'NNS': #Keine NPs, aber einzelne NN oder VB,....
                    cand.append(l1)
    w1 = taglist[-1]
    if w1.pos == 'NN'or w1.pos == 'VB' or w1.pos == 'NNS' or w1.pos == 'NP' or w1.pos == 'NNP' :
        cand.append(w1)
     
    return cand



def substantive_zaehlen(datei):
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    nouns_in_text = []   
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        for substantiv in nouns_from_sent:
            if substantiv not in nouns_in_text:
                nouns_in_text.append(substantiv)
                
    for n in nouns_in_text:
        df_n = df.get(n,0) #Der Wert von n falls n vorhanden ist, sonst 0
        df[n] = df_n + 1

for f in glob.glob("evalu-commons/*.txt"):
    substantive_zaehlen(f)

Nominalphrasen für einen Text extrahieren:

In [6]:
def extract_kw(datei,nr_of_docs):
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    nouns = [] #Liste
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True)
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        nouns.extend(nouns_from_sent)
    fdist = nltk.FreqDist(nouns)  
        
    for word in fdist:
        idf = 1.0 + math.log(float(nr_of_docs) / float(df[word]))
        fdist[word] = float(fdist[word]) / float(len(nouns)) * idf
       
    return fdist.most_common(15)
    
   
    
filelist = glob.glob("evalu-commons/*.txt")
nr_of_docs = len(filelist)
for f in filelist[0:48]:
    keywords = extract_kw(f,nr_of_docs)    
    pprint.pprint(f)
    pprint.pprint(keywords)
    pprint.pprint("------------------------------")

'evalu-commons/doi-10.1186:1471-2148-10-69-id4.txt'
[('Mason', 0.48712010109078907),
 (('Mason', 'reference'), 0.48712010109078907),
 ('manubrium', 0.40593341757565754),
 ('redrawn', 0.40593341757565754),
 ('malleus', 0.24356005054539454),
 (('mallear', 'head'), 0.16237336703026303),
 (('small', 'malleus'), 0.16237336703026303),
 ('Chlorotalpa', 0.08118668351513152),
 (('Amblysomus', 'hottentotus'), 0.08118668351513152),
 (('fossil', 'record'), 0.08118668351513152),
 ('incus', 0.08118668351513152),
 (('basal-most', 'living'), 0.08118668351513152),
 (('Chrysochloris', 'asiatica'), 0.08118668351513152),
 ('namibensis', 0.08118668351513152),
 ('chrysochlorids', 0.08118668351513152)]
'------------------------------'
'evalu-commons/doi-10.1186:1471-2148-10-69-id5.txt'
[(('temporal', 'bulla'), 0.5570738440463927),
 (('epitympanic', 'recess'), 0.32474673406052607),
 ('housing', 0.32474673406052607),
 (('dorsal', 'continuation'), 0.16237336703026303),
 (('enlarged', 'malleus'), 0.1623733670302

## Kategorien Mapping mit Auflösug der Abkürzungen
15 beste Wortpaare und/oder Terme

Nominalphrase als String ohne Sonderzeichen: ,  ) ( '  in der Funktion "def wiki_cats(term)" 
Umwandeln in Kleinbuchstaben

In [9]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper
import math
import requests
from nltk.corpus import stopwords
import string 
import re

filelist = glob.glob("evalu-commons/*.txt") #glob-Modul für Pfadnamen nach Muster
nr_of_docs = len(filelist) #Anzahl der Dateien
noun2cat = {} #Dictionary für Substantiv -> [Wikicats]
catStoplist = stopwords.words('categories') #### Stopliste Kategorien (z.B Numbers)
df = {} 
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
NPList = fdist2.keys()
acronym ={"rifsv" : "rifamycin sv", "ge" : "GE23077", "rnap" : "rna polymerase", "rif" : "rifamycin", "ic50" : "half maximal inhibitory concentrations", "sor" : "sorangicin", "rnap-sor" : "rna polymerase sorangicin", "rif-resistant" : "rifamycin resistant",
          "rpo" : "rnap promoter open complex", "rna": "ribonucleic acid", "rnap-ge" : "rna polymerase GE23077", "rnap-rif" : "RNA polymerase", "mp" : "maximum parsimony", "ghr" : "growth hormone receptor", "sro" : "spermatophore receiving organ", "rrna" : "ribosomal rna", "ml" : "maximum likelihood", "tbr" : "tree bisection and reconnection", "pca" : "principle component analysis", "pc" : "principal components", "bi" : "Bayesian inference", "pic" : "phylogenetically independent contrasts", "sro" : "spermatophore receiving organ", "mtdna" : "mitochondrial DNA", "rcrs" : "revised cambridge reference sequence", "pcr" : "polymerase chain reaction", "gtr" : "general time reversible","bpp" : "bayesian posterior probability", "mri" : "magnetic resonance imaging", "tet" : "tetrahedral", "om" : "outer membrane", "piv" : "particle image velocimetry ", "sem" : "scanning electron microscope", "rsegfp" : "reversibly switchable enhanced GFP", "gfp" : "green fluorescent protein", "egfp" : "enhanced green fluorescent protein", "paa" : "polyacrylamide layers", "er" : "endoplasmic reticulum", "bar" : "Scale Bar" }

def candidates(taglist):#Liste mit Tags, als Ausgabe als Liste
    cand = []
    skip = False
    for i in range(len(taglist)-1) :
        if skip:
            skip = False
            continue
        skip = False
        l1 = taglist[i].lemma # erstes Wort
        l2 = taglist[i+1].lemma# zweites Wort
        if len(l1) >2 and len(l2) >2:   #Die einzelnen Wörter der NP sollen eine Länge von über 2 Buchstaben haben
            if (l1,l2) in NPList:
                skip = True
                cand.append((l1,l2))
                if taglist[i].pos == 'NN' or taglist[i].pos == 'NP': #Wenn im Wortpaar ein NN vorkommt, als Term dazu
                    cand.append((l1))
            else:
                if taglist[i].pos == 'NN' or taglist[i].pos == 'VB' or taglist[i].pos == 'NNS': #Keine NPs, aber einzelne NN oder VB,....
                    cand.append(l1)
    w1 = taglist[-1]
    if w1.pos == 'NN'or w1.pos == 'VB' or w1.pos == 'NNS' or w1.pos == 'NP':
        cand.append(w1)
    
    return cand


def substantive_zaehlen(datei):
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    nouns_in_text = []   
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        for substantiv in nouns_from_sent:
            if substantiv not in nouns_in_text:
                nouns_in_text.append(substantiv)
                
    for n in nouns_in_text:
        df_n = df.get(n,0) #Der Wert von n falls n vorhanden ist, sonst 0
        df[n] = df_n + 1
    

for f in glob.glob("evalu-commons/*.txt"):
    substantive_zaehlen(f)
    
def extract_kw(datei,nr_of_docs):
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        nouns.extend(nouns_from_sent)
    fdist = nltk.FreqDist(nouns)  
    
    for word in fdist:
        idf = 1.0 + math.log(float(nr_of_docs) / float(df[word]))
        fdist[word] = float(fdist[word]) / float(len(nouns)) * idf
        
    
    return fdist.most_common(15)



def wiki_cats(term):
    cats = []    
    term_string = str(term)  #Strings, damit die Zeichen ,')( entfernt werden können
    term_lower = term_string.lower() #Kleinbuchstaben, weil zweites Wort bei Wikipedia klein beginnen muss
    rechar = re.compile(r"[',()]") #Ersetzen der Zeichen ,')(  zu einem Leerzeichen
    term_clear = rechar.sub("", term_lower)
    #pprint.pprint(term_clear) #Ansicht der bereinigten Nominalphrasen für das Mappting der Wiki-Artikel Titel
    
    if term_clear in acronym: # Abkürzungen auflösen für Mappting (Terme)
        term_clear = acronym.get(term_clear)  #Abkürzung durch aufgelöste Form ergänzen (Terme)
        
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': term_clear,  ### Bereinigte Nominalphrasen
            'prop': 'categories',
            'clshow':'!hidden',
            'cllimit':'50'
        }
    ).json()    
    #print(response)
    
    for pageid in response['query']['pages']:
        if 'categories' in response['query']['pages'][pageid]: 
            cats.extend([cat['title'] for cat in response['query']['pages'][pageid]['categories'] if cat['ns'] == 14 and not cat['title'].startswith('Category:Disambiguation') and not cat['title'].startswith('Category:Wikipedia articles incorporating') and cat['title'] not in catStoplist])
        return set(cats) #Mit set werden doppelte Vorkommen entfernt

def collect_nouns(flist):
    global noun2cat
    for f in flist:
        nouns = extract_kw(f, nr_of_docs)
        for (n,f) in nouns:   
            noun2cat[n] = wiki_cats(n)   

collect_nouns(filelist[0:48])


filelist = glob.glob("evalu-commons/*.txt")
nr_of_docs = len(filelist)
for f in filelist[0:48]:
    keywords = extract_kw(f,nr_of_docs)   
    catsPair = set([c for (kw,f) in keywords for c in noun2cat.get(kw)]) 
    pprint.pprint(f)
    pprint.pprint(catsPair)
    pprint.pprint("------------------------------")
    


'evalu-commons/doi-10.1186:1471-2148-10-69-id4.txt'
{'Category:Afrosoricida',
 'Category:Auditory system',
 'Category:Bones of the head and neck',
 'Category:Ear',
 'Category:Fossil record',
 'Category:Mammal genera',
 'Category:Mammal stubs',
 'Category:Mammals described in the 18th century'}
'------------------------------'
'evalu-commons/doi-10.1186:1471-2148-10-69-id5.txt'
{'Category:Afrosoricida',
 'Category:Anatomy stubs',
 'Category:Foramina of the skull',
 'Category:Housing',
 'Category:Mammal genera',
 'Category:Mammal stubs',
 'Category:Taxa named by St. George Jackson Mivart'}
'------------------------------'
'evalu-commons/doi-10.1186:1471-2148-10-69-id7.txt'
{'Category:Afrosoricida',
 'Category:Monotypic mammal genera',
 'Category:Types of teeth'}
'------------------------------'
'evalu-commons/doi-10.1186:1471-2148-5-25-id1.txt'
{'Category:1897 in science',
 'Category:Charge carriers',
 'Category:Cross symbols',
 'Category:Electron',
 'Category:Elementary particles',
 'Ca

### Ranking der Kategorien
Oberkategorien der Kategorien ermitteln. Anzahl der Überscheidungen = Anzahl Beziehungen
Ranking nach Anzahl

In [10]:
import requests



def wiki_cats(term):
    cats = []
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': term,
            'prop': 'categories',
            'clshow':'!hidden',
            'cllimit':'50'
        }
    ).json()
    
    for pageid in  response['query']['pages']:
        cats.extend([cat['title'] for cat in response['query']['pages'][pageid]['categories'] if cat['ns'] == 14])

    return cats # Es sind doppelte Kategorien vorhanden


def intersection(a,b):
    i = 0
    for e in a:
        if e in b:
            i += 1
    return i

def wiki_cats_sort_rel(f):
    keywords = extract_kw(f,nr_of_docs) 
    cats = set([c for (kw,f) in keywords for c in noun2cat.get(kw)]) 
    extended = {}
    relnumber = {}
    for cat in cats:
        family = [cat]
        family.extend(wiki_cats(cat))
        extended[cat] = family
    for cat in cats:
        relsize = 0
        for cat2 in cats:
            if cat == cat2:
                continue
            if intersection(extended[cat],extended[cat2]) > 0:
                relsize +=1
        relnumber[cat] = relsize
    return sorted(relnumber.items(),key = lambda x:x[1],reverse=True)
            
for f in filelist[0:48]:    
    pprint.pprint(f) 
    cts = wiki_cats_sort_rel(f)
    pprint.pprint(cts)
    pprint.pprint("------------------------------") 


'evalu-commons/doi-10.1186:1471-2148-10-69-id4.txt'
[('Category:Ear', 2),
 ('Category:Auditory system', 1),
 ('Category:Bones of the head and neck', 1),
 ('Category:Mammal stubs', 0),
 ('Category:Fossil record', 0),
 ('Category:Afrosoricida', 0),
 ('Category:Mammal genera', 0),
 ('Category:Mammals described in the 18th century', 0)]
'------------------------------'
'evalu-commons/doi-10.1186:1471-2148-10-69-id5.txt'
[('Category:Mammal stubs', 1),
 ('Category:Anatomy stubs', 1),
 ('Category:Foramina of the skull', 0),
 ('Category:Afrosoricida', 0),
 ('Category:Housing', 0),
 ('Category:Mammal genera', 0),
 ('Category:Taxa named by St. George Jackson Mivart', 0)]
'------------------------------'
'evalu-commons/doi-10.1186:1471-2148-10-69-id7.txt'
[('Category:Afrosoricida', 0),
 ('Category:Types of teeth', 0),
 ('Category:Monotypic mammal genera', 0)]
'------------------------------'
'evalu-commons/doi-10.1186:1471-2148-5-25-id1.txt'
[('Category:Leptons', 2),
 ('Category:Spintronics', 2),